In [ ]:
# %% [markdown]
# Pipeline Completo: Detecção de Mãos, Geração de Crops e Treinamento ResNet
Este notebook realiza todo o fluxo em uma única execução:
1. Instalação de dependências
2. Detecção offline de mãos com YOLOv5n
3. Geração e salvamento de crops por classe
4. Criação de `tf.data.Dataset` a partir dos crops
5. Aplicação de data augmentation
6. Definição e compilação do modelo ResNet
7. Treinamento e avaliação

In [ ]:
# %% [code]
# 1. Instalação de dependências
!pip install ultralytics tensorflow opencv-python-headless

In [ ]:
# %% [code]
# 2. Imports e configurações gerais
import os
import cv2
import numpy as np
import tensorflow as tf
from ultralytics import YOLO
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Paths (ajuste conforme sua estrutura)
DATASET_DIR = "/mnt/data/ASL_Alphabet_Dataset/asl_alphabet_train"
CROPS_DIR   = "/mnt/data/hand_crops"
IMG_SIZE    = 224
BATCH_SIZE  = 32
EPOCHS      = 20
VALID_SPLIT = 0.2
SEED        = 42

# Cria diretório para crops se não existir
os.makedirs(CROPS_DIR, exist_ok=True)

# %% [code]
# 3. Detecção e geração dos crops (por classe)
def create_hand_crops():
    # Carrega o modelo YOLOv5n pré-treinado
    model = YOLO("yolov5n.pt")
    # Itera por cada classe no dataset original
    for label in os.listdir(DATASET_DIR):
        src_dir = os.path.join(DATASET_DIR, label)
        dst_dir = os.path.join(CROPS_DIR, label)
        os.makedirs(dst_dir, exist_ok=True)
        for fname in os.listdir(src_dir):
            if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
                continue
            img_path = os.path.join(src_dir, fname)
            img = cv2.imread(img_path)
            if img is None:
                continue
            results = model(img, device='cpu')
            # Cada resultado contém múltiplas boxes
            for i, box in enumerate(results[0].boxes.xyxy.cpu().numpy()):
                x1, y1, x2, y2 = map(int, box)
                crop = img[y1:y2, x1:x2]
                if crop.size == 0:
                    continue
                out_path = os.path.join(dst_dir, f"{os.path.splitext(fname)[0]}_{i}.jpg")
                # Converte BGR -> RGB antes de salvar
                cv2.imwrite(out_path, cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))

# Executa a função de geração de crops
display("Gerando crops de mão... esta etapa pode demorar alguns minutos")
create_hand_crops()

# %% [code]
# 4. Criação dos datasets de treino e validação a partir dos crops
dataset_train = image_dataset_from_directory(
    CROPS_DIR,
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    validation_split=VALID_SPLIT,
    subset="training",
    seed=SEED
)

dataset_val = image_dataset_from_directory(
    CROPS_DIR,
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    validation_split=VALID_SPLIT,
    subset="validation",
    seed=SEED
)

# Obtém número de classes automaticamente
auto_class_count = len(dataset_train.class_names)
print(f"Número de classes detectadas: {auto_class_count}")

# %% [code]
# 5. Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomTranslation(0.1, 0.1),
])

# Aplica augmentation somente no treinamento
dataset_train = dataset_train.map(lambda x, y: (data_augmentation(x), y))
# Pré-carregamento
dataset_train = dataset_train.prefetch(tf.data.AUTOTUNE)
dataset_val   = dataset_val.prefetch(tf.data.AUTOTUNE)

# %% [code]
# 6. Definição do modelo ResNet base
base_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights=None,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = True

model = models.Sequential([
    layers.Input((IMG_SIZE, IMG_SIZE, 3)),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(auto_class_count, activation='softmax')
])
model.summary()

# %% [code]
# 7. Compilação e treinamento
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    dataset_train,
    epochs=EPOCHS,
    validation_data=dataset_val
)

# %% [code]
# 8. Avaliação final
eval_loss, eval_acc = model.evaluate(dataset_val)
print(f"Val Loss: {eval_loss:.4f}, Val Accuracy: {eval_acc:.4f}")